In [2]:
import importlib.util
import requests
import asyncio
import os
import sys
import json

from abc import ABC, abstractmethod
from typing import Optional
from pydantic import BaseModel
from fasthtml.common import List
from exa_py import Exa

/Users/devonperoutky/Development/projects/reefer/.pixi/envs/default/lib/python3.12/site-packages/bs4/css.py:8: UserWarning: The soupsieve package is not installed. CSS selectors cannot be used.
  warnings.warn(


In [3]:
class SearchResult(BaseModel):
    id: str
    url: str
    title: str
    snippet: Optional[str]


class SearchClient(ABC):
    @abstractmethod
    def search(self, query: str) -> str:
        pass


class ExaClient:
    def __init__(self) -> None:
        self.api_key = os.getenv("EXA_API_KEY")
        assert self.api_key, "EXA_API_KEY environment variable is not set."
        self.exa = Exa(self.api_key)

    def search(self, query: str) -> List[SearchResult]:
        result = self.exa.search_and_contents(
            query,
            type="neural",
            # use_autoprompt=True,
            num_results=10,
            text=True,
        )
        return [SearchResult(id=res.id, url=res.url, title=res.title, snippet=res.text) for res in result.results]


class SerpDogClient:
    def __init__(self) -> None:
        self.api_key = os.getenv("SERPDOG_API_KEY")
        assert self.api_key, "SERPDOG_API_KEY environment variable is not set."

    def search(self, query: str) -> List[SearchResult]:
        payload = {"api_key": self.api_key, "q": query, "gl": "us"}
        resp = requests.get("https://api.serpdog.io/search", params=payload)
        print(resp)
        if resp.status_code != 200:
            raise Exception(
                "Failed to fetch search results. Error: ", resp.status_code, resp.reason
            )
        return resp.text


class GoogleSearchClient:
    def __init__(self,  SEARCH_ENGINE_ID: str, API_KEY: Optional[str] = None) -> None:
        self.api_key = API_KEY or os.getenv("GOOGLE_SEARCH_API_KEY")
        self.search_engine_id = SEARCH_ENGINE_ID
        assert self.api_key, "GOOGLE_API_KEY environment variable is not set."
        assert self.search_engine_id, "GOOGLE_SEARCH_ENGINE_ID environment variable is not set."

    def search(self, query: str) -> List[SearchResult]:
        url = "https://www.googleapis.com/customsearch/v1"
        params = {
            'key': self.api_key,
            'cx': self.search_engine_id,
            'q': query
        }

        
        resp = requests.get(url, params=params)

        
        if resp.status_code != 200:
            raise Exception(
                f"Failed to fetch search results. Error: {resp.status_code} - {resp.reason}"
            )

        data = resp.json()
        print(data)
        return [
            SearchResult(
                id=str(index),
                url=item.get("link"),
                title=item.get("title"),
                snippet=item.get("snippet"),
            )
            for index, item in enumerate(data.get("items", []))
        ]

In [3]:
client = ExaClient()

In [63]:
res = client.search("valon software careers")

In [64]:
res[0]

SearchResult(id='https://isovalent.com/careers/', url='https://isovalent.com/careers/', title='Isovalent — Careers', snippet='Come join a fast growing team and help define the future of open-source networking.  See open positions    Join Isovalent Isovalent is the company founded by the creators of Cilium and eBPF. We build open source software and enterprise solutions solving networking, security, and observability needs for modern cloud native infrastructure.   Amazing technology We are driven by a simple goal: Building amazing technology. Beginning with our open source roots, building outstanding technology with a great team continues to be at the core of everything we do. We work every day to inspire our customers with our products and exceed their expectations.   Open source Building open source software is the engine of our innovation and the root of our engineering culture. We believe that open source leads to great technology and the highest quality software and we apply its be

In [11]:
google_client = GoogleSearchClient(SEARCH_ENGINE_ID='657279e3e4a414209')

In [14]:
results = google_client.search("valon careers jobs")

{'kind': 'customsearch#search', 'url': {'type': 'application/json', 'template': 'https://www.googleapis.com/customsearch/v1?q={searchTerms}&num={count?}&start={startIndex?}&lr={language?}&safe={safe?}&cx={cx?}&sort={sort?}&filter={filter?}&gl={gl?}&cr={cr?}&googlehost={googleHost?}&c2coff={disableCnTwTranslation?}&hq={hq?}&hl={hl?}&siteSearch={siteSearch?}&siteSearchFilter={siteSearchFilter?}&exactTerms={exactTerms?}&excludeTerms={excludeTerms?}&linkSite={linkSite?}&orTerms={orTerms?}&dateRestrict={dateRestrict?}&lowRange={lowRange?}&highRange={highRange?}&searchType={searchType}&fileType={fileType?}&rights={rights?}&imgSize={imgSize?}&imgType={imgType?}&imgColorType={imgColorType?}&imgDominantColor={imgDominantColor?}&alt=json'}, 'queries': {'request': [{'title': 'Google Custom Search - valon careers jobs', 'totalResults': '56100', 'searchTerms': 'valon careers jobs', 'count': 10, 'startIndex': 1, 'inputEncoding': 'utf8', 'outputEncoding': 'utf8', 'safe': 'off', 'cx': '657279e3e4a4142

In [15]:
results

[SearchResult(id='0', url='https://www.builtinnyc.com/company/valon/jobs', title='Valon NYC Jobs + Careers | Built In NYC', snippet='Explore jobs and careers at Valon in NYC. Valon is currently hiring for a range of positions.'),
 SearchResult(id='1', url='https://jobs.a16z.com/jobs/valon', title='Jobs at Valon | Andreessen Horowitz', snippet='Careers at Valon · Mortgage Assistance Processor · Client Engagement Manager · Senior Accountant · Staff Software Engineer · Business Operations Senior Associate.'),
 SearchResult(id='2', url='https://valon.com/loans/apply', title='Apply for a new home loan with Valon Mortgage', snippet='Start your application for a purchase loan, refinance, or home equity product--or, continue an existing application ... Careers · Blog · Partners. Legal. Legal\xa0...'),
 SearchResult(id='3', url='https://job-boards.greenhouse.io/valon', title='Jobs at Valon', snippet="Current openings at Valon. We're on a mission to empower every homeowner. We ... 24 jobs. Analy

In [9]:
SERPDOG_API_KEY = os.getenv("SERPDOG_API_KEY")
print(SERPDOG_API_KEY)
query = "valon careers"
payload = {"api_key": SERPDOG_API_KEY, "q": query, "gl": "us"}
# resp = requests.get("https://api.serpdog.io/search", params=payload)
resp = requests.get("http://api.scrapingdog.com/google", params=payload)
print(resp)
if resp.status_code != 200:
    raise Exception(
        "Failed to fetch search results. Error: ", resp.status_code, resp.reason
    )

63655678cd36805b2cb76220
<Response [429]>


Exception: ('Failed to fetch search results. Error: ', 429, 'Too Many Requests')